<a href="https://colab.research.google.com/github/tnalsdk111/SKN13_Sumin/blob/main/03_pytorch_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LinearRegression from Scratch


# 구현할 것
- 공부시간과 성적간의 관계를 모델링한다.
    - **머신러닝 모델(모형)이란** 수집한 데이터를 기반으로 입력값(Feature)와 출력값(Target)간의 관계를 하나의 공식으로 정의한 함수이다. 그 공식을 찾는 과정을 **모델링**이라고 한다.
    - 이 예제에서는 공부한 시험시간으로 점수를 예측하는 모델을 정의한다.
    - 입력값과 출력값 간의 관계를 정의할 수있는 다양한 함수(공식)이 있다. 여기에서는 딥러닝과 관계가 있는 **Linear Regression** 을 사용해본다.

# 데이터 확인
- 입력데이터: 공부시간
- 출력데이터: 성적

|공부시간|점수|
|-|-|
|1|20|
|2|40|
|3|60|

우리가 수집한 공부시간과 점수 데이터를 바탕으로 둘 간의 관계를 식으로 정의 할 수 있으면 **내가 몇시간 공부하면 점수를 얼마 받을 수 있는지 예측할 수 있게 된다.**   
수집한 데이터를 기반으로 앞으로 예측할 수있는 모형을 만드는 것이 머신러닝 모델링이다.

  

## 학습(훈련) 데이터셋 만들기
- 모델을 학습시키기 위한 데이터셋을 구성한다.
- 입력데이터와 출력데이터을 각각 다른 행렬로 구성한다.
- 하나의 데이터 포인트의 입력/출력 값은 같은 index에 정의한다.

### 선형회귀 (Linear Regression)
- Feature들의 가중합을 이용해 Target을 추정한다.
- Feature에 곱해지는 가중치(weight)들은 각 Feature가 Target 얼마나 영향을 주는지 영향도가 된다.
    - 음수일 경우는 target값을 줄이고 양수일 경우는 target값을 늘린다.
    - 가중치가 0에 가까울 수록 target에 영향을 주지 않는 feature이고 0에서 멀수록 target에 많은 영향을 준다.
- 모델 학습과정에서 가장 적절한 Feature의 가중치를 찾아야 한다.
      

\begin{align}
&\large \hat{y} = W\cdot X + b\\
&\small \hat{y}: \text{모델추정값}\\
&\small W: \text{가중치}\\
&\small X: \text{Feature(입력값)}\\
&\small b: \text{bias(편향)}
\end{align}



## Train dataset 구성
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `공부시간` 1개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다.   
  이 문제에서 예측할 항목은 `시험점수` 한개이다.

In [ ]:
import torch

In [ ]:
# data
study_time = [[1], [2], [3]] # 2차원으로 만들어 준다.
score = [[20], [30], [40]]

# Dataset 구성 -> torch.Tensor
X_train = torch.tensor(study_time, dtype=torch.float32)
y_train = torch.tensor(score, dtype=torch.float32)
print(X_train.size(), y_train.size())

torch.Size([3, 1]) torch.Size([3, 1])


## 파라미터 (weight, bias) 정의 (=모델 정의)
- 학습대상/최적화 대상

In [ ]:
# X * weight + bias
# 최적화 대상은 weight와 bias
torch.manual_seed(0)
# weight/bias 정의
## weight는 표준정규분포의 랜덤값을 사용한다.
weight = torch.randn(1, 1, requires_grad=True) # 최적화 대상이니까 req_grad = True로 준다.
bias = torch.randn(1, requires_grad=True) # 얘는 그냥 더하기만 하는 애니까 1만 해준다. (1, 1) 말고
print(weight)
print(bias)

tensor([[1.5410]], requires_grad=True)
tensor([-0.2934], requires_grad=True)


feature 1개니까 weight도 1개

2차원으로 (1, 1)을 만들어주자

그래야 (2, 1)과 (1, 1)은 연산 가능하니까

In [ ]:
# 모델 정의
def linear_model(X):
  return X @ weight + bias # 2 x 1 @ 1 x 1 내적

# 손실함수(Loss) 정의 -> MSE
def loss_fn(y_pred, y):
  return torch.mean((y_pred - y)**2)

torch.mean을 사용하는 이유는 PyTorch의 텐서 연산을 위해서입니다. 간단히 말하면, PyTorch 텐서에는 파이썬의 mean() 함수가 직접 적용되지 않기 때문입니다.

자세히 설명하면:

mean()은 일반적으로 파이썬의 리스트나 넘파이 배열에서 사용하는 함수입니다 (numpy.mean() 또는 statistics.mean() 등).

PyTorch의 텐서(torch.Tensor)에서 평균을 구하려면 torch.mean() 또는 tensor.mean()을 써야 합니다.

In [ ]:
y_pred = linear_model(X_train)

In [ ]:
y_pred

tensor([[1.2476],
        [2.7886],
        [4.3296]], grad_fn=<AddBackward0>)

In [ ]:
# MSE
loss = loss_fn(y_pred, y_train)
loss

tensor(788.1655, grad_fn=<MeanBackward0>)

### 모델링

### 학습
1. 모델을 이용해 추정한다.
   - pred = model(input)
1. loss를 계산한다.
   - loss = loss_fn(pred, target)
1. 계산된 loss를 파라미터에 대해 미분하여 계산한 gradient 값을 각 파라미터에 저장한다.
   - loss.backward()
1. optimizer를 이용해 파라미터를 update한다.
   - optimizer.step()  
1. 파라미터의 gradient(미분값)을 0으로 초기화한다.
   - optimizer.zero_grad()
- 위의 단계를 반복한다.   

In [ ]:
# 바꾸지 않을 상수 값은 관례적으로 대문자로 변수 생성
LEARNING_RATE = 0.01
EPOCHS = 2000 # 반복횟수

In [ ]:
for epoch in range(EPOCHS):
  # 1. 모델을 이용한 추론
  pred = linear_model(X_train) # 1 epoch
  # 2. loss 계산
  loss = loss_fn(pred, y_train)
  # 3. gradient 계산
  loss.backward() # weight/bias의 grad 값이 계산되서 각각의 grad 속성에 저장이 되어있을 것임.
  # 우리가 줄이고 싶은 건 loss니까 loss에서 backward 하는 거예요!
  # 4. 파라미터 (weight/bias) 업데이트
  weight.data = weight.data - weight.grad * LEARNING_RATE
  bias.data = bias.data - bias.grad * LEARNING_RATE
  # 5. 파라미터 grad 값 초기화 # 추론 시작 전에 해도 ㅇㅋ
  weight.grad = None
  bias.grad = None
  if epoch % 100 == 0 or epoch == EPOCHS - 1: # 100번에 한 번씩, 마지막 에폭
    print(f"[{epoch}/{EPOCHS}] loss: {loss.item()}")

[0/2000] loss: 788.16552734375
[100/2000] loss: 39.167362213134766
[200/2000] loss: 24.201217651367188
[300/2000] loss: 14.95489501953125
[400/2000] loss: 9.241212844848633
[500/2000] loss: 5.710511684417725
[600/2000] loss: 3.5287535190582275
[700/2000] loss: 2.180551767349243
[800/2000] loss: 1.3474440574645996
[900/2000] loss: 0.832639217376709
[1000/2000] loss: 0.5145217180252075
[1100/2000] loss: 0.31794390082359314
[1200/2000] loss: 0.1964704543352127
[1300/2000] loss: 0.12140478938817978
[1400/2000] loss: 0.07502076029777527
[1500/2000] loss: 0.0463578999042511
[1600/2000] loss: 0.028646698221564293
[1700/2000] loss: 0.017701977863907814
[1800/2000] loss: 0.010938555933535099
[1900/2000] loss: 0.006759307812899351
[1999/2000] loss: 0.004196911584585905


처음에는 weight 값을 엉뚱한 랜덤 값으로 하기 때문에 오차도 크고 개선도 빨리빨리 된다. 나중에는 정답에 가까워져 가니까 오차도 작고 그런거다~~

만약 처음부터 다시 학습하고 싶다면? weight 정의하는 코드를 다시 돌려야 함.

바로 위에 학습하는 for문만 한 번 더 돌리면 그냥 더 학습하는 거임

1 epoch = 전체 train data를 1회 다 학습 했을 때 1 에폭이라고 함

딥러닝에서는 전체를 1회만 학습하지 않고 여러 번 학습하기 때문에 에폭을 설정한다.

위에서 에폭 2000으로 설정 => 2000번 학습하라는 의미

In [ ]:
# 학습 후 추론
pred2 = linear_model(X_train)
pred2

tensor([[20.0956],
        [30.0205],
        [39.9455]], grad_fn=<AddBackward0>)

y값이 20, 30, 40이었으니까~! 거의 잘 맞췄네요

In [ ]:
weight, bias

(tensor([[9.9249]], requires_grad=True), tensor([10.1706], requires_grad=True))

y = 9.9249*x + 10.1706으로 추론함!

# 다중 입력, 다중 출력
- 다중입력: Feature가 여러개인 경우
- 다중출력: Output 결과가 여러개인 경우

다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의한다.  
[참조](https://www.kaggle.com/code/aakashns/pytorch-basics-linear-regression-from-scratch)


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|

```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.
        - `개별 과일를 예측하기 위한 weight들 @ feature들` 의 계산 결과를  **Node, Unit, Neuron** 이라고 한다.
        - 두 과일에 대한 Unit들을 묶어서 **Layer** 라고 한다.
- 목적은 우리가 수집한 train 데이터셋을 이용해 **정확한 예측을 위한 weight와 bias 들**을 찾는 것이다.

데이터 개수 N개

feature 개수 3개

나오는 건 사과, 오렌지 2개(출력해야할 값의 개수는 2)

(N=5, 3) * (3, 2) = (5, 2)

## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `온도, 강수량, 습도` 세개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다. 이 문제에서 예측할 항목은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [ ]:
#  input: 생산환경 (temp, rainfall, humidity) : (5, 3)
environs = [
    [73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]
]

# Targets: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = [
    [56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]
]

In [ ]:
import torch
# Dataset을 torch.Tensor로 생성
X = torch.tensor(environs, dtype=torch.float32)
y = torch.tensor(apple_orange_output, dtype=torch.float32)
X.shape, y.shape

(torch.Size([5, 3]), torch.Size([5, 2]))

In [ ]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [ ]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## weight와 bias
- weight: 각 feature들이 생산량에 영향을 주었는지의 가중치로 feature에 곱해줄 값.
    - 사과, 오렌지의 생산량을 구해야 하므로 가중치가 두개가 된다.
    - weight의 shape: `(3, 2)`
- bias는 모든 feature들이 0일때 생산량이 얼마일지를 나타내는 값으로 feature와 weight간의 가중합 결과에 더해줄 값이다.
    - 사과, 오렌지의 생산량을 구하므로 bias가 두개가 된다.
    - bias의 shape: `(2, )`

### Linear Regression model
모델은 weights `w`와 inputs `x`의 내적(dot product)한 값에 bias `b`를 더하는 함수.

$$
\hspace{2.5cm} X \hspace{1.1cm} \cdot \hspace{1.2cm} W \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\cdot
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$


<center style="font-size:0.9em">
$w_{11},\,w_{12},\,w_{13}$: 사과 생산량 계산시 각 feature들(생산환경)에 곱할 가중치   <br>
$w_{21},\,w_{22},\,w_{23}$: 오렌지 생산량 계산시 각 feature들(생산환경)에 곱할 가중치    
</center>

<center>
<img src="https://raw.githubusercontent.com/kgmyhGit/image_resource/main/deeplearning/figures/3_unit_layer.png">
</center>

In [ ]:
# weight/bias 를 정의 -> 초기값은 random 값을 이용해서 생성.
weight = torch.randn(3, 2, requires_grad=True) # 3: feature(입력) 개수, 2: 예측할 값의 개수 (출력)
bias = torch.randn(2, requires_grad=True) # 2: 예측할 값의 개수

weight.size(), bias.size()
# weight: (3:input feature개수  ,  2:output 개수)
# bias  : (2:output 개수, )

(torch.Size([3, 2]), torch.Size([2]))

In [ ]:
weight

tensor([[-0.5966,  0.1820],
        [-0.8567,  1.1006],
        [-1.0712,  0.1227]], requires_grad=True)

In [ ]:
bias

tensor([-0.5663,  0.3731], requires_grad=True)

In [ ]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [ ]:
# 한번 학습(최적화)
## 추론
pred = X @ weight + bias

In [ ]:
pred

tensor([[-147.5789,   92.6784],
        [-198.8035,  121.6445],
        [-229.3969,  170.8079],
        [-137.8941,   70.8068],
        [-198.9580,  127.1807]], grad_fn=<AddBackward0>)

In [ ]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

loss 계산 (MSE)

In [ ]:
## loss 계산(MSE)
loss = torch.mean((pred - y)**2) # 전체 추론한 결과의 평균오차를 계산.
loss

tensor(36143.9297, grad_fn=<MeanBackward0>)

사과의 loss와 오렌지의 loss를 더 합친 총 loss임.

사과를 더 잘 맞추고 싶으면: 사과의 loss를 크게 올린다. 그럼 사과 loss를 줄이려 모델이 더 잘 학습할테니까~~!!

loss를 가지고 파라미터들의 gradient 계산

In [ ]:
# loss를 가지고 파라미터들(weight들, bias들)의 gradient 계산.
loss.backward()

In [ ]:
weight.data

tensor([[-0.5966,  0.1820],
        [-0.8567,  1.1006],
        [-1.0712,  0.1227]])

In [ ]:
weight.grad

tensor([[-21555.6426,   2167.2441],
        [-24162.2188,   2128.2937],
        [-14784.2959,   1262.5558]])

In [ ]:
bias.data, bias.grad

(tensor([-0.5663,  0.3731]), tensor([-258.7263,   24.6237]))

파라미터 업데이트

In [ ]:
# 파라미터 업데이트
lr = 0.00001
weight.data = weight.data - lr * weight.grad
bias.data = bias.data - lr * bias.grad

In [ ]:
weight.data

tensor([[-0.3811,  0.1604],
        [-0.6151,  1.0793],
        [-0.9233,  0.1101]])

In [ ]:
## 업데이트된 파라미터로 추정 -> loss 계산
pred2 = X @ weight + bias
loss2 = torch.mean((pred2 - y)**2)

In [ ]:
print(loss.item(), loss2.item())

36143.9296875 24508.60546875


36000 -> 24000 많이 줄었죠?

##  모델링

In [ ]:
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

## 모델 정의
def model(X):
    return X @ weight + bias

## loss 함수(MSE)
def loss_fn(pred, y):
    return torch.mean((pred - y)**2) # 전체 오차의 평균.

In [ ]:
X.shape

torch.Size([5, 3])

In [ ]:
epochs = 5000
lr = 0.00001  # 1e-5
for epoch in range(epochs):
    # 1. 추론
    pred = model(X)

    # 2. loss 계산
    loss = loss_fn(pred, y)
    # 3. 파라미터 들의 gradient 계산
    loss.backward()
    # 4. 파라미터 업데이트
    weight.data = weight.data - lr * weight.grad
    bias.data = bias.data - lr * bias.grad
    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None
    ## 100 epoch, 마지막 epoch에서 loss를 출력 => 학습 과정 log를 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item():.5f}")

[0001/5000] - 4102.97900
[0101/5000] - 167.19608
[0201/5000] - 52.97421
[0301/5000] - 19.95751
[0401/5000] - 9.89216
[0501/5000] - 6.40350
[0601/5000] - 4.87352
[0701/5000] - 3.98703
[0801/5000] - 3.35902
[0901/5000] - 2.86814
[1001/5000] - 2.46935
[1101/5000] - 2.14086
[1201/5000] - 1.86902
[1301/5000] - 1.64368
[1401/5000] - 1.45678
[1501/5000] - 1.30173
[1601/5000] - 1.17311
[1701/5000] - 1.06641
[1801/5000] - 0.97788
[1901/5000] - 0.90443
[2001/5000] - 0.84350
[2101/5000] - 0.79295
[2201/5000] - 0.75101
[2301/5000] - 0.71622
[2401/5000] - 0.68735
[2501/5000] - 0.66340
[2601/5000] - 0.64354
[2701/5000] - 0.62705
[2801/5000] - 0.61338
[2901/5000] - 0.60203
[3001/5000] - 0.59262
[3101/5000] - 0.58481
[3201/5000] - 0.57834
[3301/5000] - 0.57296
[3401/5000] - 0.56850
[3501/5000] - 0.56480
[3601/5000] - 0.56173
[3701/5000] - 0.55919
[3801/5000] - 0.55707
[3901/5000] - 0.55532
[4001/5000] - 0.55386
[4101/5000] - 0.55266
[4201/5000] - 0.55166
[4301/5000] - 0.55083
[4401/5000] - 0.55014
[45

In [ ]:
# 새로운 데이터로 추론
p = model(X)
p

tensor([[ 57.1935,  70.5227],
        [ 82.2194, 100.5309],
        [118.6196, 132.9357],
        [ 21.0584,  36.9917],
        [101.9937, 119.1602]], grad_fn=<AddBackward0>)

In [ ]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

학습하는 데이터셋이 2차원이였기 때문에 추론할 데이터셋도 차원을 맞춰줘야 한다.

-> unsqueeze를 써서 2차원으로 늘려준다.

(3,) -> (1, 3)

In [ ]:
new_x = torch.tensor([68, 82, 56], dtype=torch.float32)
new_x = new_x.unsqueeze(dim=0)
new_x.shape

torch.Size([1, 3])

In [ ]:
model(new_x)

tensor([[80.8797, 95.7239]], grad_fn=<AddBackward0>)

# pytorch built-in 모델을 사용해 Linear Regression 구현

In [ ]:
inputs = torch.tensor(
    [[73, 67, 43],
     [91, 88, 64],
     [87, 134, 58],
     [102, 43, 37],
     [69, 96, 70]], dtype=torch.float32)

In [ ]:
targets = torch.tensor(
    [[56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]], dtype=torch.float32)

## torch.nn.Linear
Pytorch는 torch.nn.Linear 클래스를 통해 Linear Regression 모델을 제공한다.  
torch.nn.Linear에 입력 feature의 개수와 출력 값의 개수를 지정하면 random 값으로 초기화한 weight와 bias들을 생성해 모델을 구성한다.
- `torch.nn.Linear(input feature의 개수 , output 값의 개수)`

## Optimizer와 Loss 함수 정의
- **Optimizer**: 계산된 gradient값을 이용해 파라미터들을 업데이트 하는 함수
- **Loss 함수**: 정답과 모델이 예측한 값사이의 차이(오차)를 계산하는 함수.
  - 모델을 최적화하는 것은 이 함수의 값을 최소화하는 것을 말한다.
- `torch.optim` 모듈에 다양한 Optimizer 클래스가 구현되있다.
- `torch.nn` 또는 `torch.nn.functional` 모듈에 다양한 Loss 함수가 제공된다.

In [ ]:
# 선형회귀 모델을 정의. torch.nn.Linear 클래스
import torch
import torch.nn as nn

model = nn.Linear(3, 2)  # 3: input feature 개수, 2: output 수

In [ ]:
# loss 함수
loss_fn = torch.nn.MSELoss()  # 클래스
# loss_fn = torch.nn.functional.mse_loss # 함수

SGD: Stocastic Gradient Descent

In [ ]:
# optimizer (torch.optim 모듈에 정의): weight.data = weight.data - lr * weight.grad
optimizer = torch.optim.SGD(
    model.parameters(), # 최적화 대상 파라미터들을 model에서 조회해서 전달.
    lr = 0.00001,       # Learning Rage
)

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2291,  0.0780,  0.3871],
         [-0.3399,  0.1076, -0.4476]], requires_grad=True),
 Parameter containing:
 tensor([-0.4002, -0.2982], requires_grad=True)]

2 x 3짜리가 weight, 밑에 두 개가 bias

optimizer가 업데이트하니까 optimizer한테 model.parameters() 넘겨주는 거임

## Model Train

In [ ]:
epochs = 5000

for epoch in range(epochs):
    # 추론
    pred = model(inputs)
    # loss 계산
    loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)
    # gradient 계산
    loss.backward()
    # 파라미터 업데이트: optimizer.step()
    optimizer.step()
    # 파라미터 초기화 w.grad=None, b.grad=None
    optimizer.zero_grad()
    # 현재 epoch 학습 결과를 log로 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

[0001/5000] - 10742.8310546875
[0101/5000] - 151.3299102783203
[0201/5000] - 63.65910720825195
[0301/5000] - 35.50872039794922
[0401/5000] - 24.664966583251953
[0501/5000] - 19.179325103759766
[0601/5000] - 15.613412857055664
[0701/5000] - 12.93144702911377
[0801/5000] - 10.783990859985352
[0901/5000] - 9.024163246154785
[1001/5000] - 7.570270538330078
[1101/5000] - 6.365779876708984
[1201/5000] - 5.3669939041137695
[1301/5000] - 4.538535118103027
[1401/5000] - 3.8512444496154785
[1501/5000] - 3.2810630798339844
[1601/5000] - 2.8080222606658936
[1701/5000] - 2.4155800342559814
[1801/5000] - 2.0899860858917236
[1901/5000] - 1.81988525390625
[2001/5000] - 1.5957990884780884
[2101/5000] - 1.4098782539367676
[2201/5000] - 1.2556408643722534
[2301/5000] - 1.1276843547821045
[2401/5000] - 1.021523356437683
[2501/5000] - 0.9334546327590942
[2601/5000] - 0.8603876233100891
[2701/5000] - 0.7997709512710571
[2801/5000] - 0.749481201171875
[2901/5000] - 0.707760214805603
[3001/5000] - 0.673148751

In [ ]:
# 추론 => gradient 계산을 할 필요가 없다. ==> grad_fn을 만들 필요가 없다. 그래서 torch.no_grad() 블록에서 추론 작업을 실행한다.
with torch.no_grad():
    pred = model(inputs)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [ ]:
pred

tensor([[ 57.0985,  70.2679],
        [ 82.2408, 100.6381],
        [118.7217, 133.0966],
        [ 21.0950,  37.0579],
        [101.8919, 119.0000]])

train하는 게 반복적이니까 함수로 구현

In [ ]:
# 학습 로직을 함수 구현
def train(inputs, targets, epochs, model, loss_fn, optimizer):

    for epoch in range(epochs):
        # 추론
        pred = model(inputs)
        # loss 계산
        loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)
        # gradient 계산
        loss.backward()
        # 파라미터 업데이트: optimizer.step()
        optimizer.step()
        # 파라미터 초기화 w.grad=None, b.grad=None
        optimizer.zero_grad()
        # 현재 epoch 학습 결과를 log로 출력
        if epoch % 100 == 0 or epoch == epochs-1:
            print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

In [ ]:
model = nn.Linear(3, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [ ]:
train(inputs, targets, 5000, model, nn.functional.mse_loss, optimizer)

[0001/5000] - 3782.52587890625
[0101/5000] - 10.657798767089844
[0201/5000] - 2.071007251739502
[0301/5000] - 0.7552521228790283
[0401/5000] - 0.5536100268363953
[0501/5000] - 0.5227049589157104
[0601/5000] - 0.5179613828659058
[0701/5000] - 0.5172356367111206
[0801/5000] - 0.5171126127243042
[0901/5000] - 0.5170959830284119
[1001/5000] - 0.5170840620994568
[1101/5000] - 0.5170764327049255
[1201/5000] - 0.5170726776123047
[1301/5000] - 0.5170679688453674
[1401/5000] - 0.5170631408691406
[1501/5000] - 0.5170561075210571
[1601/5000] - 0.517051100730896
[1701/5000] - 0.5170449614524841
[1801/5000] - 0.5170422792434692
[1901/5000] - 0.5170327425003052
[2001/5000] - 0.5170321464538574
[2101/5000] - 0.5170232653617859
[2201/5000] - 0.5170175433158875
[2301/5000] - 0.517012894153595
[2401/5000] - 0.5170049667358398
[2501/5000] - 0.517004132270813
[2601/5000] - 0.516995370388031
[2701/5000] - 0.5169912576675415
[2801/5000] - 0.5169833898544312
[2901/5000] - 0.5169780850410461
[3001/5000] - 0.5